# Business Problem

## For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

Over the course of this project, you will:

- Part 1: Download several files from IMDB’s movie data set and filter out the subset of moves requested by the stakeholder.
- Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.
- Part 3: Construct and export a MySQL database using your data.
- Part 4: Apply hypothesis testing to explore what makes a movie successful.
- Part 5 (Optional): Produce a Linear Regression model to predict movie performance.

##  Imports

In [1]:
import pandas as pd
import numpy as np

# The Data

- IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.

- Overview/Data Dictionary: https://www.imdb.com/interfaces/
- Downloads page: https://datasets.imdbws.com/
- From their previous research, they realized they want to focus on the following files:

- title.basics.tsv.gz
- title.ratings.tsv.gz
- title.akas.tsv.gz

# Load in the Data

In [2]:
# set up urls
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [3]:
# read in the data
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# Database Basics

## Filtering Cleaning

- Filtering/Cleaning Steps:
- Title Basics:
- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- keep only titleType==Movie
- keep startYear 2000-2022
- Eliminate movies that include "Documentary" in genre (see tip below)
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [4]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            15
dtype: int64

In [5]:
basics = basics.replace({"\\N":np.nan})
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1335669
endYear           9804235
runtimeMinutes    6977471
genres             444466
dtype: int64

# Eliminate the movies that have null for runtime

In [6]:
basics = basics.dropna(subset = ["runtimeMinutes"], axis = 0)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 1
startYear          173813
endYear           2883135
runtimeMinutes          0
genres              76861
dtype: int64

In [7]:
basics.dropna(subset = ["genres"], axis = 0, inplace = True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          168878
endYear           2807872
runtimeMinutes          0
genres                  0
dtype: int64

In [8]:
# This line of code keeps only titletype Movie  
basics = basics[basics["titleType"] == "movie"]
basics["titleType"].info()

<class 'pandas.core.series.Series'>
Int64Index: 383533 entries, 8 to 9912583
Series name: titleType
Non-Null Count   Dtype 
--------------   ----- 
383533 non-null  object
dtypes: object(1)
memory usage: 5.9+ MB


In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383533 entries, 8 to 9912583
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          383533 non-null  object
 1   titleType       383533 non-null  object
 2   primaryTitle    383533 non-null  object
 3   originalTitle   383533 non-null  object
 4   isAdult         383533 non-null  object
 5   startYear       377061 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  383533 non-null  object
 8   genres          383533 non-null  object
dtypes: object(9)
memory usage: 29.3+ MB


In [10]:
basics.dropna(subset = ["startYear"], axis = 0, inplace = True)
basics["startYear"] = basics["startYear"].astype(dtype = int) 
basics = basics[(basics["startYear"] >= 2000) & (basics["startYear"] <= 2022)]

In [11]:
basics["startYear"].describe()

count    223620.000000
mean       2013.373428
std           5.854230
min        2000.000000
25%        2009.000000
50%        2014.000000
75%        2018.000000
max        2022.000000
Name: startYear, dtype: float64

In [12]:
# This line of code eLiminates Movies with Documentary in genre
is_documentary = basics["genres"].str.contains("documentary", case=False)
basics = basics[~is_documentary]

In [13]:
basics["genres"].value_counts()

Drama                        36078
Comedy                       13463
Comedy,Drama                  6463
Horror                        5804
Drama,Romance                 4316
                             ...  
Family,Musical,Sport             1
Horror,Music,Mystery             1
Comedy,History,Mystery           1
Animation,Biography,Sport        1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 955, dtype: int64

## AKAS Database

In [14]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36139188 entries, 0 to 36139187
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [15]:
# This line keeps only the US movies in it
akas = akas[akas["region"] == "US"]
akas["region"].value_counts()

US    1443212
Name: region, dtype: int64

In [16]:
akas = akas.replace({"\\N":np.nan})
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1439267
types               463400
attributes         1396509
isOriginalTitle       1342
dtype: int64

In [17]:
keepers = basics["tconst"].isin(akas["titleId"])
basics = basics[keepers]

## Ratings Database

In [18]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320058 entries, 0 to 1320057
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1320058 non-null  object 
 1   averageRating  1320058 non-null  float64
 2   numVotes       1320058 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.2+ MB


In [19]:
ratings = ratings.replace({"\\N":np.nan})
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [20]:
keepers =ratings["tconst"].isin(akas["titleId"])
ratings = ratings[keepers]

## Verify Info

In [21]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86753 entries, 34803 to 9912399
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86753 non-null  object
 1   titleType       86753 non-null  object
 2   primaryTitle    86753 non-null  object
 3   originalTitle   86753 non-null  object
 4   isAdult         86753 non-null  object
 5   startYear       86753 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86753 non-null  object
 8   genres          86753 non-null  object
dtypes: int32(1), object(8)
memory usage: 6.3+ MB


In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501007 entries, 0 to 1320033
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         501007 non-null  object 
 1   averageRating  501007 non-null  float64
 2   numVotes       501007 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


In [23]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1443212 entries, 5 to 36138932
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1443212 non-null  object
 1   ordering         1443212 non-null  int64 
 2   title            1443212 non-null  object
 3   region           1443212 non-null  object
 4   language         3945 non-null     object
 5   types            979812 non-null   object
 6   attributes       46703 non-null    object
 7   isOriginalTitle  1441870 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.1+ MB


# Saving the Files

In [24]:
import os
os.makedirs('Data/',exist_ok=True) 

#This line confirms folder created
os.listdir("Data/")

['title_basics.csv.gz']

In [25]:
basics.to_csv("Data/title_basics.csv.gz",compression="gzip",index=False)

# Open up the saved file and preview it
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [26]:
akas.to_csv("Data/title_akas.csv.gz",compression="gzip",index=False)

# Open up the saved file and preview it
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [27]:
ratings.to_csv("Data/title_ratings.csv.gz",compression="gzip",index=False)

# Open up the saved file and preview it
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1978
1,tt0000002,5.8,265
2,tt0000005,6.2,2621
3,tt0000006,5.1,182
4,tt0000007,5.4,821
